In [ ]:
import pandas as pd

df = pd.read_csv("/content/2024schedule.csv")
df.head()

,Date,Num,Day,Visitor,League,Game,Home,League.1,Game.1,Day/Night,Location,Postponed,Makeup
0,20240320,0,Wednesday,LAN,NL,1,SDN,NL,1,n,SEO01,NaN,NaN
1,20240321,0,Thursday,SDN,NL,2,LAN,NL,2,n,SEO01,NaN,NaN
2,20240328,0,Thursday,MIL,NL,1,NYN,NL,1,d,NYC20,Rain,20240329
3,20240328,0,Thursday,ANA,AL,1,BAL,AL,1,d,BAL12,NaN,NaN
4,20240328,0,Thursday,ATL,NL,1,PHI,NL,1,d,PHI13,Rain,20240329


In [ ]:
df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')


In [ ]:
df.head()

,Date,Num,Day,Visitor,League,Game,Home,League.1,Game.1,Day/Night,Location,Postponed,Makeup
0,2024-03-20,0,Wednesday,LAN,NL,1,SDN,NL,1,n,SEO01,NaN,NaN
1,2024-03-21,0,Thursday,SDN,NL,2,LAN,NL,2,n,SEO01,NaN,NaN
2,2024-03-28,0,Thursday,MIL,NL,1,NYN,NL,1,d,NYC20,Rain,20240329
3,2024-03-28,0,Thursday,ANA,AL,1,BAL,AL,1,d,BAL12,NaN,NaN
4,2024-03-28,0,Thursday,ATL,NL,1,PHI,NL,1,d,PHI13,Rain,20240329


In [ ]:
!pip install airportsdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.4/914.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
from airportsdata import load
from math import radians, sin, cos, sqrt, atan2

# ----------------------------------------------
# 1. MLB → City mapping
# ----------------------------------------------
MLB_TO_CITY = {
    'ANA': 'Los Angeles',
    'ARI': 'Phoenix',
    'ATL': 'Atlanta',
    'BAL': 'Baltimore',
    'BOS': 'Boston',
    'CHA': 'Chicago',       # White Sox
    'CHN': 'Chicago',       # Cubs
    'CIN': 'Cincinnati',
    'CLE': 'Cleveland',
    'COL': 'Denver',
    'DET': 'Detroit',
    'HOU': 'Houston',
    'KCA': 'Kansas City',
    'LAN': 'Los Angeles',   # Dodgers
    'MIA': 'Miami',
    'MIL': 'Milwaukee',
    'MIN': 'Minneapolis',
    'NYA': 'New York',      # Yankees
    'NYN': 'New York',      # Mets
    'OAK': 'Oakland',
    'PHI': 'Philadelphia',
    'PIT': 'Pittsburgh',
    'SDN': 'San Diego',
    'SEA': 'Seattle',
    'SFN': 'San Francisco',
    'SLN': 'St. Louis',
    'TBA': 'Tampa',
    'TEX': 'Dallas',
    'TOR': 'Toronto',
    'WAS': 'Washington'
}

# ----------------------------------------------
# 2. City → Primary Airport mapping
# ----------------------------------------------
CITY_TO_AIRPORT = {
    'New York': 'JFK',
    'Los Angeles': 'LAX',
    'Chicago': 'ORD',
    'Dallas': 'DFW',
    'Tampa': 'TPA',
    'Miami': 'MIA',
    'Philadelphia': 'PHL',
    'Houston': 'IAH',
    'Atlanta': 'ATL',
    'Phoenix': 'PHX',
    'Denver': 'DEN',
    'San Francisco': 'SFO',
    'San Diego': 'SAN',
    'Oakland': 'OAK',
    'Seattle': 'SEA',
    'St. Louis': 'STL',
    'Milwaukee': 'MKE',
    'Minneapolis': 'MSP',
    'Cleveland': 'CLE',
    'Cincinnati': 'CVG',
    'Detroit': 'DTW',
    'Pittsburgh': 'PIT',
    'Baltimore': 'BWI',
    'Boston': 'BOS',
    'Kansas City': 'MCI',
    'Toronto': 'YYZ',
    'Washington': 'DCA'
}

# ----------------------------------------------
# 3. Haversine function
# ----------------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))


# ----------------------------------------------
# 4. One-shot enrichment function
# ----------------------------------------------
def enrich_with_distance(df, home_col='Home', away_col='Visitor'):
    airports = load('IATA')

    # Map MLB → city
    df['home_city'] = df[home_col].map(MLB_TO_CITY)
    df['away_city'] = df[away_col].map(MLB_TO_CITY)

    # Map city → airport
    df['home_airport'] = df['home_city'].map(CITY_TO_AIRPORT)
    df['away_airport'] = df['away_city'].map(CITY_TO_AIRPORT)

    # Resolve coordinates
    df['home_lat']  = df['home_airport'].apply(lambda x: airports[x]['lat'])
    df['home_lon']  = df['home_airport'].apply(lambda x: airports[x]['lon'])
    df['away_lat']  = df['away_airport'].apply(lambda x: airports[x]['lat'])
    df['away_lon']  = df['away_airport'].apply(lambda x: airports[x]['lon'])

    # Compute haversine distance
    df['distance_km'] = df.apply(
        lambda r: haversine(r['home_lat'], r['home_lon'], r['away_lat'], r['away_lon']),
        axis=1
    )

    return df

In [ ]:
df2 = enrich_with_distance(df, home_col='Home', away_col='Visitor')

df2['distance_km'] = round(df2['distance_km'],2)
df2.head()

,Date,Num,Day,Visitor,League,Game,Home,League.1,Game.1,Day/Night,...,Makeup,home_city,away_city,home_airport,away_airport,home_lat,home_lon,away_lat,away_lon,distance_km
0,2024-03-20,0,Wednesday,LAN,NL,1,SDN,NL,1,n,...,NaN,San Diego,Los Angeles,SAN,LAX,32.73356,-117.18966,33.94250,-118.40805,175.73
1,2024-03-21,0,Thursday,SDN,NL,2,LAN,NL,2,n,...,NaN,Los Angeles,San Diego,LAX,SAN,33.94250,-118.40805,32.73356,-117.18966,175.73
2,2024-03-28,0,Thursday,MIL,NL,1,NYN,NL,1,d,...,20240329,New York,Milwaukee,JFK,MKE,40.63993,-73.77869,42.94693,-87.89706,1196.67
3,2024-03-28,0,Thursday,ANA,AL,1,BAL,AL,1,d,...,NaN,Baltimore,Los Angeles,BWI,LAX,39.17573,-76.66899,33.94250,-118.40805,3739.86
4,2024-03-28,0,Thursday,ATL,NL,1,PHI,NL,1,d,...,20240329,Philadelphia,Atlanta,PHL,ATL,39.87208,-75.24066,33.63670,-84.42786,1071.62


In [ ]:
df2 = df2.drop(['home_lat', 'home_lon', 'away_lat', 'away_lon'], axis = 1)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430 entries, 0 to 2429
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          2430 non-null   datetime64[ns]
 1   Num           2430 non-null   int64         
 2   Day           2430 non-null   object        
 3   Visitor       2430 non-null   object        
 4   League        2430 non-null   object        
 5   Game          2430 non-null   int64         
 6   Home          2430 non-null   object        
 7   League.1      2430 non-null   object        
 8   Game.1        2430 non-null   int64         
 9   Day/Night     2430 non-null   object        
 10  Location      2430 non-null   object        
 11  Postponed     35 non-null     object        
 12  Makeup        34 non-null     object        
 13  home_city     2430 non-null   object        
 14  away_city     2430 non-null   object        
 15  home_airport  2430 non-null   object  

# Post Cleaning df

In [ ]:
df2['Home'].unique()

array(['SDN', 'LAN', 'NYN', 'BAL', 'PHI', 'CHA', 'CIN', 'HOU', 'KCA',
       'MIA', 'TBA', 'TEX', 'OAK', 'ARI', 'SEA', 'CHN', 'WAS', 'MIL',
       'MIN', 'SLN', 'NYA', 'DET', 'COL', 'PIT', 'SFN', 'ATL', 'ANA',
       'TOR', 'CLE', 'BOS'], dtype=object)

In [ ]:
df2['Visitor'].unique()

array(['LAN', 'SDN', 'MIL', 'ANA', 'ATL', 'DET', 'WAS', 'NYA', 'MIN',
       'PIT', 'SFN', 'TOR', 'SLN', 'CHN', 'CLE', 'COL', 'BOS', 'KCA',
       'CIN', 'TEX', 'MIA', 'CHA', 'OAK', 'TBA', 'BAL', 'NYN', 'PHI',
       'ARI', 'HOU', 'SEA'], dtype=object)

In [ ]:
df2.Postponed.value_counts()

,count
Postponed,
Rain,33
20240616,1
Hurricane Helene,1


In [ ]:
import pandas as pd
import numpy as np

# 1. Filter to clean, single games
df_clean = df2[
    (df2['Num'] == 0) &
    df2['Postponed'].isna() &
    df2['Makeup'].isna()
].copy()

# 2. Convert Date from yyyymmdd to datetime
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='mixed')

# 3. Sort so we can look at consecutive games
df_clean = df_clean.sort_values(['Home', 'Visitor', 'Date'])

# 4. Within each (Home, Visitor), flag the start of a new series
#    A new series starts when:
#    - it's the first game for that home/visitor pair, OR
#    - the gap from the previous game is > 1 day
df_clean['date_diff'] = (
    df_clean.groupby(['Home', 'Visitor'])['Date']
            .diff()
            .dt.days
)

df_clean['new_series'] = (
    df_clean['date_diff'].isna() | (df_clean['date_diff'] > 1)
)

# 5. Create a series_id by cumulatively summing the "new_series" flags
df_clean['series_id'] = (
    df_clean.groupby(['Home', 'Visitor'])['new_series']
            .cumsum()
)

# (optional) drop helper columns later if you like
# df_clean = df_clean.drop(columns=['date_diff', 'new_series'])

# 6. Get series-level counts (# of games per series)
series_sizes = (
    df_clean
    .groupby(['Home', 'Visitor', 'series_id'])
    .size()
    .rename('games_in_series')
    .reset_index()
)

# 7. How many 2-game, 3-game, etc. series?
series_length_counts = (
    series_sizes['games_in_series']
    .value_counts()
    .sort_index()
)

print(series_length_counts)


games_in_series
1     42
2     64
3    587
4    115
Name: count, dtype: int64


In [ ]:
df_clean['Date'] = pd.to_datetime(df_clean['Date'])


In [ ]:
# 1. Filter to clean, single games
df_clean = df2[
    (df2['Num'] == 0) &
    df2['Postponed'].isna() &
    df2['Makeup'].isna()
].copy()

# 2. Convert ISO-format dates
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# 3. Sort
df_clean = df_clean.sort_values(['Home', 'Visitor', 'Date'])

# 4. Compute date gaps within each (Home, Visitor)
df_clean['date_diff'] = (
    df_clean.groupby(['Home', 'Visitor'])['Date']
            .diff()
            .dt.days
)

df_clean['new_series'] = (
    df_clean['date_diff'].isna() | (df_clean['date_diff'] > 1)
)

# 5. Series ID
df_clean['series_id'] = (
    df_clean.groupby(['Home', 'Visitor'])['new_series']
            .cumsum()
)

# 6. Size of each series
series_sizes = (
    df_clean.groupby(['Home', 'Visitor', 'series_id'])
            .size()
            .rename('games_in_series')
            .reset_index()
)

# 7. Count frequencies (2-game, 3-game, etc.)
series_length_counts = (
    series_sizes['games_in_series']
    .value_counts()
    .sort_index()
)

series_length_counts


,count
games_in_series,
1,42
2,64
3,587
4,115


In [ ]:
import pandas as pd
import numpy as np

# 0. Start from your original df

# 1. Filter: only clean, scheduled single games for now (your test condition)
df_clean = df2[
    (df2['Num'] == 0) &
    df2['Postponed'].isna() &
    df2['Makeup'].isna()
].copy()

# 2. Make sure Date is datetime
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# 3. Sort so we can look at games in order *within the same park*
df_clean = df_clean.sort_values(['Home', 'Visitor', 'Location', 'Date'])

# 4. Within each (Home, Visitor, Location), compute the gap in days
df_clean['date_diff'] = (
    df_clean.groupby(['Home', 'Visitor', 'Location'])['Date']
            .diff()
            .dt.days
)

# 5. A new series starts when:
#    - it's the first game in that (Home, Visitor, Location) group, OR
#    - the gap from the previous game > 1 day
df_clean['new_series'] = df_clean['date_diff'].isna() | (df_clean['date_diff'] > 1)

# 6. Build a series_id by cumulatively summing new_series
#    Do this within (Home, Visitor, Location) so each ballpark pairing has its own sequence.
df_clean['series_id'] = (
    df_clean.groupby(['Home', 'Visitor', 'Location'])['new_series']
            .cumsum()
)

# 7. Series-level summary: how many games per series?
series_sizes = (
    df_clean
      .groupby(['Home', 'Visitor', 'Location', 'series_id'])
      .size()
      .rename('games_in_series')
      .reset_index()
)

# 8. Distribution of series lengths (MLB-style)
series_length_counts = (
    series_sizes['games_in_series']
      .value_counts()
      .sort_index()
)

print(series_length_counts)


games_in_series
1     43
2     65
3    586
4    115
Name: count, dtype: int64


# How do we incorporate makeup games correctly?

In [ ]:
df_clean = df2.copy()
df_clean['Date'] = pd.to_datetime(df_clean['Date'])


In [ ]:
df_clean

,Date,Num,Day,Visitor,League,Game,Home,League.1,Game.1,Day/Night,Location,Postponed,Makeup,home_city,away_city,home_airport,away_airport,distance_km
0,2024-03-20,0,Wednesday,LAN,NL,1,SDN,NL,1,n,SEO01,NaN,NaN,San Diego,Los Angeles,SAN,LAX,175.73
1,2024-03-21,0,Thursday,SDN,NL,2,LAN,NL,2,n,SEO01,NaN,NaN,Los Angeles,San Diego,LAX,SAN,175.73
2,2024-03-28,0,Thursday,MIL,NL,1,NYN,NL,1,d,NYC20,Rain,20240329,New York,Milwaukee,JFK,MKE,1196.67
3,2024-03-28,0,Thursday,ANA,AL,1,BAL,AL,1,d,BAL12,NaN,NaN,Baltimore,Los Angeles,BWI,LAX,3739.86
4,2024-03-28,0,Thursday,ATL,NL,1,PHI,NL,1,d,PHI13,Rain,20240329,Philadelphia,Atlanta,PHL,ATL,1071.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,2024-09-29,0,Sunday,NYN,NL,162,MIL,NL,162,d,MIL06,NaN,NaN,Milwaukee,New York,MKE,JFK,1196.67
2426,2024-09-29,0,Sunday,BAL,AL,162,MIN,AL,162,d,MIN04,NaN,NaN,Minneapolis,Baltimore,MSP,BWI,1503.55
2427,2024-09-29,0,Sunday,OAK,AL,162,SEA,AL,162,d,SEA03,NaN,NaN,Seattle,Oakland,SEA,OAK,1081.80
2428,2024-09-29,0,Sunday,KCA,AL,162,ATL,NL,162,d,ATL03,NaN,NaN,Atlanta,Kansas City,ATL,MCI,1113.39


In [ ]:
df_clean['effective_date'] = df_clean['Date']

# If a makeup game exists (Makeup column filled),
# 'Date' is the actual play date already.
# If Postponed is filled, keep original date for intended schedule version.


In [ ]:
df_actual = df_clean.sort_values(['Home', 'Visitor', 'Location', 'Date'])

df_actual['date_diff'] = (
    df_actual.groupby(['Home','Visitor','Location'])['Date']
             .diff().dt.days
)

df_actual['new_series'] = df_actual['date_diff'].isna() | (df_actual['date_diff'] > 1)

df_actual['actual_series_id'] = (
    df_actual.groupby(['Home','Visitor','Location'])['new_series']
             .cumsum()
)


In [ ]:
df_actual

,Date,Num,Day,Visitor,League,Game,Home,League.1,Game.1,Day/Night,...,Makeup,home_city,away_city,home_airport,away_airport,distance_km,effective_date,date_diff,new_series,actual_series_id
1839,2024-08-16,0,Friday,ATL,NL,124,ANA,AL,122,n,...,NaN,Los Angeles,Atlanta,LAX,ATL,3125.80,2024-08-16,NaN,True,1
1854,2024-08-17,0,Saturday,ATL,NL,125,ANA,AL,123,n,...,NaN,Los Angeles,Atlanta,LAX,ATL,3125.80,2024-08-17,1.0,False,1
1867,2024-08-18,0,Sunday,ATL,NL,126,ANA,AL,124,d,...,NaN,Los Angeles,Atlanta,LAX,ATL,3125.80,2024-08-18,1.0,False,1
343,2024-04-22,0,Monday,BAL,AL,22,ANA,AL,23,n,...,NaN,Los Angeles,Baltimore,LAX,BWI,3739.86,2024-04-22,NaN,True,1
358,2024-04-23,0,Tuesday,BAL,AL,23,ANA,AL,24,n,...,NaN,Los Angeles,Baltimore,LAX,BWI,3739.86,2024-04-23,1.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2024-07-07,0,Sunday,SLN,NL,91,WAS,NL,90,d,...,NaN,Washington,St. Louis,DCA,STL,1154.39,2024-07-07,1.0,False,1
1363,2024-07-08,0,Monday,SLN,NL,92,WAS,NL,91,d,...,NaN,Washington,St. Louis,DCA,STL,1154.39,2024-07-08,1.0,False,1
481,2024-05-03,0,Friday,TOR,AL,33,WAS,NL,32,n,...,NaN,Washington,Toronto,DCA,YYZ,578.63,2024-05-03,NaN,True,1
497,2024-05-04,0,Saturday,TOR,AL,34,WAS,NL,33,d,...,NaN,Washington,Toronto,DCA,YYZ,578.63,2024-05-04,1.0,False,1


In [ ]:
# Make sure Date is already datetime
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# 1. Try to parse Makeup as a date; invalid values (like 'none') become NaT
makeup_parsed = pd.to_datetime(df_clean['Makeup'], errors='coerce')

# 2. Intended date = Makeup date when valid, otherwise the actual Date
df_clean['intended_date'] = makeup_parsed.fillna(df_clean['Date'])


In [ ]:
# Sort by intended schedule
df_intended = df_clean.sort_values(['Home','Visitor','Location','intended_date'])

# Compute date gaps within each home/visitor/location
df_intended['date_diff'] = (
    df_intended.groupby(['Home','Visitor','Location'])['intended_date']
               .diff()
               .dt.days
)

# New series when first game or gap > 1 day
df_intended['new_series'] = df_intended['date_diff'].isna() | (df_intended['date_diff'] > 1)

# Intended series ID
df_intended['intended_series_id'] = (
    df_intended.groupby(['Home','Visitor','Location'])['new_series']
               .cumsum()
)

# How many games per intended series?
intended_series_sizes = (
    df_intended
      .groupby(['Home','Visitor','Location','intended_series_id'])
      .size()
      .rename('games_in_intended_series')
      .reset_index()
)

intended_series_length_counts = (
    intended_series_sizes['games_in_intended_series']
      .value_counts()
      .sort_index()
)

intended_series_length_counts


,count
games_in_intended_series,
1,36
2,59
3,592
4,125


# Incoprorating double headers

In [ ]:
df_clean = df2.copy()
df_clean['Date'] = pd.to_datetime(df_clean['Date'])
df_clean = df_clean.sort_values(['Home','Visitor','Location','Date','Num'])


In [ ]:
df_clean['date_diff'] = (
    df_clean.groupby(['Home','Visitor','Location'])['Date']
            .diff()
            .dt.days
)

df_clean['new_series'] = df_clean['date_diff'].isna() | (df_clean['date_diff'] > 1)

df_clean['actual_series_id'] = (
    df_clean.groupby(['Home','Visitor','Location'])['new_series']
            .cumsum()
)


In [ ]:
series_sizes = (
    df_clean
      .groupby(['Home','Visitor','Location','actual_series_id'])
      .size()                       # this counts how many rows (games)
      .rename('games_in_series')
      .reset_index()
)


In [ ]:
series_sizes['games_in_series'].value_counts().sort_index()


,count
games_in_series,
1,18
2,52
3,604
4,124


In [ ]:
df_clean

,Date,Num,Day,Visitor,League,...,away_airport,distance_km,date_diff,new_series,actual_series_id
1839,2024-08-16,0,Friday,ATL,NL,...,ATL,3125.80,NaN,True,1
1854,2024-08-17,0,Saturday,ATL,NL,...,ATL,3125.80,1.0,False,1
1867,2024-08-18,0,Sunday,ATL,NL,...,ATL,3125.80,1.0,False,1
343,2024-04-22,0,Monday,BAL,AL,...,BWI,3739.86,NaN,True,1
358,2024-04-23,0,Tuesday,BAL,AL,...,BWI,3739.86,1.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...
1350,2024-07-07,0,Sunday,SLN,NL,...,STL,1154.39,1.0,False,1
1363,2024-07-08,0,Monday,SLN,NL,...,STL,1154.39,1.0,False,1
481,2024-05-03,0,Friday,TOR,AL,...,YYZ,578.63,NaN,True,1
497,2024-05-04,0,Saturday,TOR,AL,...,YYZ,578.63,1.0,False,1


# Creating a new df for series

In [ ]:
series_df = (
    df_clean
      .groupby(['Home', 'Visitor', 'Location', 'actual_series_id'])
      .agg(
          series_start=('Date', 'min'),
          series_end=('Date', 'max'),
          games_in_series=('Date', 'size'),
          home_league=('League.1', 'first'),
          visitor_league=('League', 'first'),
          home_city=('home_city', 'first'),
          away_city=('away_city', 'first'),
          distance_km=('distance_km', 'first'),
          # example: how many doubleheaders in the series
          # n_doubleheader_games=('Num', lambda s: (s != 0).sum()),
      )
      .reset_index()
)


In [ ]:
series_df

,Home,Visitor,Location,actual_series_id,series_start,series_end,games_in_series,home_league,visitor_league,home_city,away_city,distance_km
0,ANA,ATL,ANA01,1,2024-08-16,2024-08-18,3,AL,NL,Los Angeles,Atlanta,3125.80
1,ANA,BAL,ANA01,1,2024-04-22,2024-04-24,3,AL,AL,Los Angeles,Baltimore,3739.86
2,ANA,BOS,ANA01,1,2024-04-05,2024-04-07,3,AL,AL,Los Angeles,Boston,4192.94
3,ANA,CHA,ANA01,1,2024-09-16,2024-09-18,3,AL,AL,Los Angeles,Chicago,2801.82
4,ANA,CLE,ANA01,1,2024-05-24,2024-05-26,3,AL,AL,Los Angeles,Cleveland,3296.01
...,...,...,...,...,...,...,...,...,...,...,...,...
793,WAS,SDN,WAS11,1,2024-07-23,2024-07-25,3,NL,NL,Washington,San Diego,3654.77
794,WAS,SEA,WAS11,1,2024-05-24,2024-05-26,3,NL,AL,Washington,Seattle,3738.38
795,WAS,SFN,WAS11,1,2024-08-05,2024-08-08,4,NL,NL,Washington,San Francisco,3921.00
796,WAS,SLN,WAS11,1,2024-07-05,2024-07-08,4,NL,NL,Washington,St. Louis,1154.39


# Incorporating distance between home and away team (realistically)

In [39]:
import numpy as np
import pandas as pd

all_series = series_df.copy()

# Rename the visitor→home1 distance column
all_series = all_series.rename(columns={'distance_km': 'dist_visitorhome_to_home1'})

# Sort
all_series = all_series.sort_values(['Visitor', 'series_start'])



all_series['next_home_team']    = all_series.groupby('Visitor')['Home'].shift(-1)
all_series['next_home_city']    = all_series.groupby('Visitor')['home_city'].shift(-1)
all_series['next_series_start'] = all_series.groupby('Visitor')['series_start'].shift(-1)



city_dist_lookup = (
    df2
      .dropna(subset=['home_city', 'away_city', 'distance_km'])
      .assign(
          city_pair=lambda d: d.apply(
              lambda r: tuple(sorted([r['home_city'], r['away_city']])),
              axis=1
          )
      )
      .groupby('city_pair')['distance_km']
      .mean()
      .to_dict()
)



def compute_home1_to_home2(row):
    if pd.isna(row['next_home_city']):
        return np.nan
    cp = tuple(sorted([row['home_city'], row['next_home_city']]))
    return city_dist_lookup.get(cp, np.nan)

all_series['dist_home1_to_home2'] = all_series.apply(compute_home1_to_home2, axis=1)



# Total distance for that series
all_series['series_distance'] = (
    all_series['dist_visitorhome_to_home1'].fillna(0) +
    all_series['dist_home1_to_home2'].fillna(0)
)

# Half-distance per series
all_series['series_distance_half'] = all_series['series_distance'] / 2



all_series = (
    all_series
      .sort_values(['Visitor', 'series_start'])
      .assign(
          cumulative_distance=lambda d:
              d.groupby('Visitor')['series_distance'].cumsum()
      )
)



team_totals = (
    all_series
      .groupby('Visitor')['series_distance']
      .sum()
      .reset_index()
      .rename(columns={'series_distance': 'total_distance'})
)

# Also compute DISTANCE per team (very explicit)
team_totals['total_distance_one_month'] = round(team_totals['total_distance'] / 6 , 2)
team_totals['total_distance_one_month_half'] = round(team_totals['total_distance_one_month'] / 2 , 2)
# Sort by total distance (most to least)
team_totals = team_totals.sort_values('total_distance', ascending=False)


team_totals


,Visitor,total_distance,total_distance_one_month,total_distance_one_month_half
24,SFN,111517.26,18586.21,9293.10
19,OAK,109727.95,18287.99,9144.00
23,SEA,105719.30,17619.88,8809.94
1,ARI,104130.38,17355.06,8677.53
0,ANA,103763.52,17293.92,8646.96
13,LAN,101468.77,16911.46,8455.73
14,MIA,95471.29,15911.88,7955.94
22,SDN,94703.23,15783.87,7891.94
11,HOU,90296.76,15049.46,7524.73
27,TEX,85659.42,14276.57,7138.28


# Last Checkpoint

In [ ]:
# Build a lookup: (cityA, cityB) -> average distance between them
city_dist_lookup = (
    df2
      .dropna(subset=['home_city', 'away_city', 'distance_km'])
      .assign(
          city_pair=lambda d: d.apply(
              lambda r: tuple(sorted([r['home_city'], r['away_city']])),
              axis=1
          )
      )
      .groupby('city_pair')['distance_km']
      .mean()
      .to_dict()
)

def debug_team_travel(team, start=None, end=None):
    """
    Debug travel distances for ONE team (AL or NL) using all_series.
    Shows stored vs recomputed distances side-by-side.
    """
    # pick this team's rows from your ALL-series table
    sub = all_series[all_series['Visitor'] == team].copy()
    sub = sub.sort_values('series_start')

    if start is not None:
        sub = sub[sub['series_start'] >= pd.to_datetime(start)]
    if end is not None:
        sub = sub[sub['series_start'] <= pd.to_datetime(end)]

    if sub.empty:
        print(f"No rows found for team {team} in that date range.")
        return

    # recompute distances from df2
    def recompute(row):
        # visitor home -> home1: directly from df2 for that (Home, Visitor)
        d1_raw = (
            df2[
                (df2['Home'] == row['Home']) &
                (df2['Visitor'] == row['Visitor'])
            ]['distance_km']
            .mean()
        )

        # home1 -> home2: via city_dist_lookup using home_city + next_home_city
        if pd.isna(row['next_home_city']):
            d2_raw = np.nan
        else:
            cp = tuple(sorted([row['home_city'], row['next_home_city']]))
            d2_raw = city_dist_lookup.get(cp, np.nan)

        return pd.Series({'d1_raw': d1_raw, 'd2_raw': d2_raw})

    checks = sub.apply(recompute, axis=1)
    sub = pd.concat([sub, checks], axis=1)

    # show stored vs recomputed side-by-side
    cols_to_show = [
        'Visitor', 'Home', 'series_start',
        'home_city', 'next_home_city',
        'dist_visitorhome_to_home1', 'd1_raw',
        'dist_home1_to_home2', 'd2_raw',
        'series_distance', 'cumulative_distance'
    ]

    print(sub[cols_to_show].to_string(index=False))


In [ ]:
# Build a lookup: (cityA, cityB) -> average distance between them
city_dist_lookup = (
    df2
      .dropna(subset=['home_city', 'away_city', 'distance_km'])
      .assign(
          city_pair=lambda d: d.apply(
              lambda r: tuple(sorted([r['home_city'], r['away_city']])),
              axis=1
          )
      )
      .groupby('city_pair')['distance_km']
      .mean()
      .to_dict()
)

def debug_team_travel(team, start=None, end=None):
    """
    Debug travel distances for ONE team (AL or NL) using all_series.
    Shows stored vs recomputed distances side-by-side.
    """
    # pick this team's rows from your ALL-series table
    sub = all_series[all_series['Visitor'] == team].copy()
    sub = sub.sort_values('series_start')

    if start is not None:
        sub = sub[sub['series_start'] >= pd.to_datetime(start)]
    if end is not None:
        sub = sub[sub['series_start'] <= pd.to_datetime(end)]

    if sub.empty:
        print(f"No rows found for team {team} in that date range.")
        return

    # recompute distances from df2
    def recompute(row):
        # visitor home -> home1: directly from df2 for that (Home, Visitor)
        d1_raw = (
            df2[
                (df2['Home'] == row['Home']) &
                (df2['Visitor'] == row['Visitor'])
            ]['distance_km']
            .mean()
        )

        # home1 -> home2: via city_dist_lookup using home_city + next_home_city
        if pd.isna(row['next_home_city']):
            d2_raw = np.nan
        else:
            cp = tuple(sorted([row['home_city'], row['next_home_city']]))
            d2_raw = city_dist_lookup.get(cp, np.nan)

        return pd.Series({'d1_raw': d1_raw, 'd2_raw': d2_raw})

    checks = sub.apply(recompute, axis=1)
    sub = pd.concat([sub, checks], axis=1)

    # show stored vs recomputed side-by-side
    cols_to_show = [
        'Visitor', 'Home', 'series_start',
        'home_city', 'next_home_city',
        'dist_visitorhome_to_home1', 'd1_raw',
        'dist_home1_to_home2', 'd2_raw',
        'series_distance', 'cumulative_distance'
    ]

    print(sub[cols_to_show].to_string(index=False))


In [ ]:
debug_team_travel('ANA')
debug_team_travel('PIT', start='2024-04-01', end='2024-05-31')

Visitor Home series_start     home_city next_home_city  dist_visitorhome_to_home1  d1_raw  dist_home1_to_home2  d2_raw  series_distance  cumulative_distance
    ANA  BAL   2024-03-28     Baltimore      Baltimore                    3739.86 3739.86                  NaN     NaN          3739.86              3739.86
    ANA  BAL   2024-03-30     Baltimore          Miami                    3739.86 3739.86              1525.72 1525.72          5265.58              9005.44
    ANA  MIA   2024-04-01         Miami         Boston                    3762.58 3762.58              2027.86 2027.86          5790.44             14795.88
    ANA  BOS   2024-04-12        Boston          Tampa                    4192.94 4192.94              1908.28 1908.28          6101.22             20897.10
    ANA  TBA   2024-04-15         Tampa     Cincinnati                    3466.43 3466.43              1246.99 1246.99          4713.42             25610.52
    ANA  CIN   2024-04-19    Cincinnati      Cleveland    